In [1]:
!git clone https://github.com/Mike030668/Project_glass.git -q

In [2]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  import os
  os.chdir('/content/Project_glass')
  os.getcwd()

else:
  print('Not running on CoLab')

Running on CoLab


In [10]:
from prepare_data.constants import GLASS_COLS, FUTURE, MAIN_COLS, DEPTH, PREDICT_LAG, BATCH_SIZE, MAKE_LOG_VOL
from prepare_data.const_predprice import  TARGET_COLS, BASE_COLS, MAKE_LOG_TARGET
from prepare_data.utils import future_sequence, split_sequence, clean_dataset
import tensorflow as tf  # библиотека машинного обучения
import pandas as pd # библиотека пандас
import numpy as np # библиотека нампи
import joblib
from tqdm.notebook import tqdm   # отрисовка прохождения цикла
import warnings # библиотека сообщений по ошибкам
warnings.filterwarnings("ignore") # игнорировать сообщения ошибок

## Готовим производные данных

In [4]:
all_data = pd.read_csv("/content/Project_glass/stock_data/BTCUSDT_1 _min.csv",
                       index_col=0, parse_dates=True )
print(f'Размер данных {all_data.shape}')

TEST_SIZE = 3500 # для теста откусим последние test_size значений
# Тестовая выборка для симуляции торговли в реальном времени
#print(f'Тестовый датасет')
#TEST_DATASET = all_data[FUTURE: FUTURE + TEST_SIZE]
#print(TEST_DATASET.shape)


Размер данных (8317, 54)


In [5]:
# колонки для тренировочной выборки
TREND_PRED_COLS = MAIN_COLS + GLASS_COLS

# колонки для тренировочной выборки
PRICE_PRED_COLS = BASE_COLS + GLASS_COLS

GLASS = len(GLASS_COLS)

In [6]:
trend_model =  tf.keras.models.load_model('./predtrend/model')
price_model =  tf.keras.models.load_model('./predprice/model')

x_trend_main_scr = joblib.load("/content/Project_glass/predtrend/x_main_scailer.save")
x_trend_glass_scr = joblib.load("/content/Project_glass/predtrend/x_glass_scailer.save")
trend_scalers = [x_trend_main_scr, x_trend_glass_scr]

x_price_main_scr = joblib.load("/content/Project_glass/predprice/x_main_scailer.save")
x_price_glass_scr = joblib.load("/content/Project_glass/predprice/x_glass_scailer.save")
y_price_scr = joblib.load("/content/Project_glass/predprice/y_scailer.save")
price_scalers = [x_price_main_scr, x_price_glass_scr, y_price_scr]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [25]:
def prepare_data(df_in: pd.DataFrame,
                        depth:int,
                        make_log,
                        scalers,
                        glass,
                        features
                      ):
    # обогащаем данные ранее созданной функцией
    ds = df_in.copy()
    if make_log:
       ds["Volume"] = ds["Volume"].apply(lambda x: np.log(x))
    # Удаление строк с неполными данными
    ds = clean_dataset(ds)
    # оставляем на глубину depth и только нужные колонки
    x_data = ds[features][-depth:].values
    # нормализуем
    x_out = np.zeros_like(x_data)
    x_out[:,:-glass] = scalers[0].transform(x_data[:,:-glass])  # трансформируем x_train
    x_out[:,-glass:] = scalers[1].transform(x_data[:,-glass:])  # трансформируем x_train

    # выводим с добавлением измерения по axis=0
    return np.expand_dims(x_out, axis=0)



def making_signals(past_df: pd.DataFrame,
                    check_df: pd.DataFrame,
                    pred_lag: int,
                    glass: int,
                    model_price: object,
                    model_trend: object,
                    scalers_price: object,
                    scalers_trend: object,
                    features_price: list,
                    features_trend: list,
                    control_enter = False,
                    stop_loss = 1.,
                    use_force_action = False,
                    show_unique_signals = True,
                    ):

    """
    Args:
        past_df (pd.DataFrame) - прошлые OHCL данные до тестируемых
        check_df (pd.DataFrame) - тестируемые OHCL данные
        pred_lag: int - шаг предсказания
        model (class) - ранее обученная модель
        control_enter (bool) - контролировать ли вход в сделку
        stop_loss (float) - уровень падения цены (от 0 до 1) при которм нужно продать
        use_force_action (bool): False - выводим чистые действия,
                                 True - выводим действия с учетом уверенности,
        show_unique_signals (bool) - показать ли состав чистых действий

    Return:
         df_signal(pd.DataFrame) - копия OHCL датафрейма из check_df с колонкой Signal
    """

    # сбор действий
    all_actions = []
    # сбор уверенности действия
    force_actions = []
    # сбор последнего активного действия и цены
    last_position = [0, 0]
    # итеррируемся по длине изучаемого датафрейма
    # берем на страте прошлые данные
    #print("metka_1")
    for i in tqdm(range(check_df.shape[0]-pred_lag), unit ="step",
                      desc ="Пробегаемся по всем отсчетам"):
        if not i: # берем на страте прошлые данные
            check_data = past_df.copy()
        # далее отшипываем вначале 1 свечу
        else: #  и присоединяем в коней новую из изучаемого датафрейма
            check_data = check_data[1:].append(check_df[i:i+1])
        #print("metka_2")
        # обогащаем данные по аналогии как готовили для убучени
        to_pred_price = prepare_data(df_in = check_data,
                                      depth = DEPTH ,    # ранее заданная глубина сбора данных в прошлое
                                      make_log = MAKE_LOG_VOL,
                                      scalers = scalers_price,
                                      glass = glass,
                                      features = features_price
                                      )
        #print("metka_3")
        # предсказание модели
        pred_price = model_price.predict(to_pred_price, verbose=False)
        #print("metka_4")
        pred_price = scalers_price[2].inverse_transform(pred_price)
        if MAKE_LOG_TARGET: pred_price = np.exp(pred_price)
        #print("pred_price.shape", pred_price.shape)

         # обогащаем данные по аналогии как готовили для убучени
        to_pred_trend = prepare_data(df_in = check_data,
                                      depth = DEPTH ,    # ранее заданная глубина сбора данных в прошлое
                                      make_log = MAKE_LOG_VOL,
                                      scalers = scalers_trend,
                                      glass = GLASS,
                                      features = features_trend
                                      )
        #print("metka_5")
        # предсказание модели
        pred_trend = model_trend.predict(to_pred_trend, verbose=False)
        #print("pred_trend.shape", pred_trend.shape)
        price = check_data.Close[-1]

        if pred_trend.mean() > 0 and pred_price.mean() > price:
           print(1)
        elif pred_trend.mean() < - 0 and pred_price.mean() < price:
           print(-1)
        else:
          print(0)
        pass

In [26]:
# длина взгляда назад
BACK = 2*DEPTH + PREDICT_LAG
TEST_SIZE = 100
# данные назад
past_data =  all_data[FUTURE-BACK : FUTURE + TEST_SIZE].copy()

# тестируемы датафрейм
test_data = all_data[FUTURE-PREDICT_LAG : FUTURE + TEST_SIZE].copy()

# получаем датафрейм с сигналом
df_test_signal = making_signals(past_data,
                                test_data,
                                pred_lag = PREDICT_LAG,
                                glass = GLASS,
                                model_price = price_model,
                                model_trend = trend_model,
                                scalers_price = price_scalers,
                                scalers_trend = trend_scalers,
                                features_price = PRICE_PRED_COLS,
                                features_trend = TREND_PRED_COLS,
                                control_enter = True,
                                stop_loss = 0.8,
                                use_force_action = False,
                                show_unique_signals = True
                                )

Пробегаемся по всем отсчетам:   0%|          | 0/100 [00:00<?, ?step/s]

0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
0
0
-1
0
-1
0
0
0
0
0
0
0
0
0
0
1
0
1
-1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
